In [149]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier


In [150]:
test = pd.read_csv('test_df.csv')

In [151]:
data = pd.read_csv('train_df.csv')
data.target.value_counts()

target
0    14759
1      322
Name: count, dtype: int64

Присутствуается дисбаланс классов

In [152]:
data

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
0,758,9,0,0,1,20,3,40,0,3,...,0.204682,0.271755,0.055623,0,0,0,0.38648,0.0,0.0,0
1,758,9,0,0,1,20,3,40,0,3,...,0.195531,0.188787,0.036914,0,0,0,0.10982,0.0,0.0,0
2,758,9,0,0,1,20,3,40,0,3,...,0.148609,0.186517,0.027718,0,0,0,0.03674,0.0,0.0,0
3,758,9,0,0,1,20,3,40,0,3,...,0.223748,0.229039,0.051247,0,0,0,0.00000,0.0,0.0,0
4,758,9,0,0,1,20,3,40,0,3,...,0.170935,0.249031,0.042568,0,0,0,0.00000,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15076,494693,9,0,0,0,9,4,38,6,6,...,0.309672,0.921060,0.285226,0,0,0,0.98807,0.0,0.0,0
15077,494693,9,0,0,0,9,4,38,6,6,...,0.303805,0.995086,0.302312,0,0,0,0.87146,0.0,0.0,0
15078,494693,9,0,0,0,9,4,38,6,6,...,0.346538,0.993070,0.344137,0,0,0,0.49999,0.0,0.0,0
15079,494693,9,0,0,0,9,4,38,6,6,...,0.243154,0.994833,0.241898,0,0,0,0.67614,0.0,0.0,0


## Data Preprocessing

In [153]:
data.describe()

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
count,15081.000000,15081.0,15081.000000,15081.000000,15081.000000,15081.000000,15081.000000,15081.000000,15081.000000,15081.000000,...,15081.000000,15081.000000,15081.000000,15081.0,15081.0,15081.0,15081.000000,15081.000000,15081.000000,15081.000000
mean,254806.827465,9.0,0.193754,0.464293,0.629865,15.928519,3.876401,33.802467,1.211193,1.793847,...,0.199545,0.426543,0.096461,0.0,0.0,0.0,0.297465,0.030543,0.031207,0.021351
std,142374.568081,0.0,0.626654,1.183076,0.482857,5.311422,0.469749,11.316810,2.234887,1.885131,...,0.088724,0.247388,0.063323,0.0,0.0,0.0,0.311962,0.112984,0.115598,0.144558
min,116.000000,9.0,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,134823.000000,9.0,0.000000,0.000000,0.000000,9.000000,4.000000,26.000000,0.000000,0.000000,...,0.168583,0.255086,0.052658,0.0,0.0,0.0,0.036740,0.000000,0.000000,0.000000
50%,255859.000000,9.0,0.000000,0.000000,1.000000,20.000000,4.000000,34.000000,0.000000,2.000000,...,0.212743,0.430880,0.091705,0.0,0.0,0.0,0.176140,0.000000,0.000000,0.000000
75%,376175.000000,9.0,0.000000,0.000000,1.000000,20.000000,4.000000,44.000000,1.000000,3.000000,...,0.253825,0.603686,0.136017,0.0,0.0,0.0,0.529100,0.000000,0.000000,0.000000
max,496975.000000,9.0,3.000000,6.000000,1.000000,20.000000,6.000000,49.000000,7.000000,7.000000,...,0.485510,0.999072,0.411563,0.0,0.0,0.0,1.000020,0.846272,0.908132,1.000000


Некоторые переменные не несут никакую смысловую нагрузку (например все 9ки или нолики), их убираем + смотри на сильно коррелированные данные

In [154]:
corr_matrix = data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

In [155]:
data.drop(columns = to_drop + ['feature_0', 'search_id', 'feature_73', 'feature_74', 'feature_75'], inplace=True)
test.drop(columns = to_drop + ['feature_0', 'search_id', 'feature_73', 'feature_74', 'feature_75'], inplace=True)

In [156]:
x_train = data.drop(['target'], axis=1)
y_train = data.target

x_test = test.drop(['target'], axis=1)
y_test = test.target

In [157]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train),
                                              y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

In [158]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Modelling

Так как у нас есть дисбаланс классов, было решено смотреть на roc-auc как на метрику. 

AdaBoostClassifier показал высокий результат на roc-auc, поэтому с помошь GridSearch с cv=5, scroing='roc-auc' и sample_weight были выявлены лучшие параметры.

In [159]:
# param_grid = {
#     'n_estimators': [25, 50, 100, 200],
#     'learning_rate': [0.01, 0.1, 0.5, 1.0],
#     'random_state': [42],
# }

# adaboost_classifier = AdaBoostClassifier()

# grid_search = GridSearchCV(adaboost_classifier, param_grid, scoring='roc_auc', cv=5)
# grid_search.fit(x_train, y_train, sample_weight=np.array([class_weight_dict[label] for label in y_train]))
# best_params = grid_search.best_params_

# best_adaboost_classifier = AdaBoostClassifier(**best_params).fit(x_train, y_train, sample_weight=np.array([class_weight_dict[label] for label in y_train]))

# adaboost_pred = best_adaboost_classifier.predict(x_test)

# roc_auc = roc_auc_score(y_test, adaboost_pred)
# accuracy = accuracy_score(y_test, adaboost_pred)

# print("Best Parameters:", best_params)
# print("ROC AUC Score:", roc_auc)
# print("Accuracy Score:", accuracy)


GridSearchCV показал плохие результаты на других моделях, поэтому было решено оставить дефолтные параметры.

In [160]:
parametrs = {'learning_rate': 0.01, 'n_estimators': 100, 'random_state': 42}
adaboost = AdaBoostClassifier(**parametrs).fit(x_train, y_train, sample_weight=np.array([class_weight_dict[label] for label in y_train]))
adaboost_pred = adaboost.predict(x_test)
print('ROCAUC score:',roc_auc_score(y_test, adaboost_pred))
print("Accuracy score:", accuracy_score(y_test, adaboost_pred))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


ROCAUC score: 0.7278575644304546
Accuracy score: 0.7488554610856769


Так как на других моделях было замечено, что ROC-AUC примерно от 0.5-0.6, но точность было 0.9, было инетерсно сделать ансамбль из лучших моделей, чтобы за получить за счет небольшого уменьшения roc-auc точность выше.

Было замечено, что у LinearSVC был неплохой ROCAUC и Accuracy с помощью GridSearch параметров, а с дефолтным получился чуть выше 0.5 (уже нелохой результат, учитвая, что большинство моделей давали 0.5) и неплохой accuracy. Было решено использовать эти 2 модели.

In [161]:
# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'penalty': ['l1', 'l2'],
#     'loss': ['hinge', 'squared_hinge'],
#     'dual': [True, False],
#     'max_iter': [1000, 2000, 3000]
# }

# svc_classifier = LinearSVC()

# grid_search = GridSearchCV(svc_classifier, param_grid, scoring='roc_auc', cv=5)
# grid_search.fit(x_train, y_train, sample_weight=np.array([class_weight_dict[label] for label in y_train]))
# best_params = grid_search.best_params_

# best_svc_classifier = LinearSVC(**best_params).fit(x_train, y_train, sample_weight=np.array([class_weight_dict[label] for label in y_train]))

# svc_pred = best_svc_classifier.predict(x_test)

# roc_auc = roc_auc_score(y_test, svc_pred)
# accuracy = accuracy_score(y_test, svc_pred)

# print("Best Parameters:", best_params)
# print("ROC AUC Score:", roc_auc)
# print("Accuracy Score:", accuracy)

In [162]:
parametrs = {'C': 0.1, 'dual': True, 'loss': 'squared_hinge', 'max_iter': 1000, 'penalty': 'l2'}
svc = LinearSVC(**parametrs)
svc.fit(x_train, y_train, sample_weight=np.array([class_weight_dict[label] for label in y_train]))
svc_pred = svc.predict(x_test)
print('ROCAUC score:',roc_auc_score(y_test, svc_pred))
print("Accuracy score:", accuracy_score(y_test, svc_pred))

ROCAUC score: 0.6813791068266771
Accuracy score: 0.6860693263570962


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [163]:
from sklearn.svm import LinearSVC 
svc_default = LinearSVC()
svc_default.fit(x_train, y_train, sample_weight=np.array([class_weight_dict[label] for label in y_train]))
svc_default_pred = svc_default.predict(x_test)
print('ROCAUC score:', roc_auc_score(y_test, svc_default_pred))
print("Accuracy score:", accuracy_score(y_test, svc_default_pred))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


ROCAUC score: 0.5658272673617942
Accuracy score: 0.9378678875081753


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Получив 3 модели, используем VotingClassifier:

In [164]:
voting_classifier = VotingClassifier(
    estimators=[('ada', adaboost), ('svc_dr', svc_default), ('svc', svc)],
    voting='hard'
)

voting_classifier.fit(x_train, y_train, sample_weight=np.array([class_weight_dict[label] for label in y_train]))

voting_pred = voting_classifier.predict(x_test)
accuracy = accuracy_score(y_test, voting_pred)
roc_auc = roc_auc_score(y_test, voting_pred)

print(f"Ensemble ROCAUC score: {roc_auc}")
print(f"Ensemble Accuracy score: {accuracy}")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Ensemble ROCAUC score: 0.7068266771591579
Ensemble Accuracy score: 0.8201438848920863


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


С помощью ансамбля получилось повысить accuracy на 0.08 за счет потери в roc-auc 0.02

В работе с данными, так же было испробовано PolynomialFeatures and PCA, но результаты Adaboostа были хуже.

Проблема дисбаланса классов так же пыталась быть решена с over-sampling and under-sampling (так же разные умные алгоритмы типа SMOTE), что не привело к успеху. Самым эффективным оказалось использовать sample_weight во время обучения модели